# 11장 항공 사진 내 선인장 식별 경진대회 환경 세팅된 노트북 양식

In [ ]:
import pandas as pd

labels = pd.read_csv('/kaggle/input/aerial-cactus-identification/train.csv')
submission = pd.read_csv('/kaggle/input/aerial-cactus-identification/sample_submission.csv')
#submission.head()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

mpl.rc('font', size=15)
plt.figure(figsize=(7,7))
label=['Has cactus','hasn\'t cactus']
plt.pie(labels['has_cactus'].value_counts(), labels=label, autopct='%0.1f%%')

In [ ]:
# 이미지 출력 483
from zipfile import ZipFile

with ZipFile('/kaggle/input/aerial-cactus-identification/train.zip') as zipper:
    zipper.extractall()

with ZipFile('/kaggle/input/aerial-cactus-identification/test.zip') as zipper:
    zipper.extractall()

In [ ]:
import os
# print(len(os.listdir('train/')))
# print(len(os.listdir('test/')))

# 이미지 출력해보기 485

import matplotlib.gridspec as gridspec
import cv2

mpl.rc('font', size=7)
plt.figure(figsize=(15,6))
grid = gridspec.GridSpec(2,6)

last_has_cactus_img_name = labels[labels['has_cactus']==1]['id'][-12:]

for idx, img_name in enumerate(last_has_cactus_img_name):
    img_path = 'train/'+img_name
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax = plt.subplot(grid[idx])
    ax.imshow(img)

In [ ]:
# 선인장이 아닌 사진 출력하기
plt.figure(figsize=(15,6))
grid = gridspec.GridSpec(2,6)

last_has_cactus_img_name = labels[labels['has_cactus']==0]['id'][-12:]

for idx, img_name in enumerate(last_has_cactus_img_name):
    img_path = 'train/'+img_name
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # 주석처리 해보세요. 칼라 읽는 방법을 변경한 코드입니다.
    ax = plt.subplot(grid[idx])
    ax.imshow(img)

In [ ]:
# 489
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.daterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [ ]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu')

In [ ]:
# 데이터 처리 494
import pandas as pd

labels = pd.read_csv('/kaggle/input/aerial-cactus-identification/train.csv')
submission = pd.read_csv('/kaggle/input/aerial-cactus-identification/sample_submission.csv')


from zipfile import ZipFile

with ZipFile('/kaggle/input/aerial-cactus-identification/train.zip') as zipper:
    zipper.extractall()

with ZipFile('/kaggle/input/aerial-cactus-identification/test.zip') as zipper:
    zipper.extractall()

In [ ]:
#데이터셋 정의하기 495~
import cv2
from torch.utils.data import Dataset

class ImageDataset(Dataset):
    # 초기화 메소드
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    # 데이터셋 크기 반환 메소드
    def __len__(self):
        return len(self.df)

    # 인덱스에 해당하는 데이터 반환 메소드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image=cv2.imgred(img_path)
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[idx,1]


        if self.transform is not None:
            image = self.transform(image)
            return image, label

In [ ]:
# 498
from torchvision import transforms
transform = transform.ToTensor()

dataset_train = ImageDataset(df = train, img_dir="train/", transform = transform)
dataset_valid = ImageDataset(df = valid, img_dir="train/", transform = transform)

dataset_train.__len__()
dataset_valid.__len__()

In [ ]:
# 데이터 로더 만들기 499

from torch.utils.data import DataLoader
loader_train = DataLoader(dataset = dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=32, shuffle=True)

In [ ]:
# 모델 만들기 501
import torch.nn as nn # 신경망 모듈 사용하기
import torch.nn.functional as F # 함수. 대문자로 표시하겠습니다.

# class
class Model(nn.Module): # 신경망 모델
    # 초기화
    def __init__(self):
        super().__inif__() # nn.Model 초기화 메소드 호출
        self.conv1 = nn.Conv2d(in_channels=3, out_chnnels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=3, out_chnnels=32, kernel_size=3, padding=2)
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=2)
        self.fc = nn.Linear(in_features=64*4*4, out_features=2)

    # 순전파 출력
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.svg_pool(x)
        x = x.view(-1, 64 * 4 * 4)
        x = self.fc(x)
        return x

In [ ]:
# 490
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
device


# 데이터 처리 494
import pandas as pd

labels = pd.read_csv('/kaggle/input/aerial-cactus-identification/train.csv')
submission = pd.read_csv('/kaggle/input/aerial-cactus-identification/sample_submission.csv')


from zipfile import ZipFile

with ZipFile('/kaggle/input/aerial-cactus-identification/train.zip') as zipper:
    zipper.extractall()

with ZipFile('/kaggle/input/aerial-cactus-identification/test.zip') as zipper:
    zipper.extractall()



from sklearn.model_selection import train_test_split

train, valid = train_test_split(labels, 
                                test_size=0.1, 
                                stratify=labels['has_cactus'],
                                random_state=50)




#데이터셋 정의하기 495~
import cv2
from torch.utils.data import Dataset

class ImageDataset(Dataset):
    # 초기화 메소드
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    # 데이터셋 크기 반환 메소드
    def __len__(self):
        return len(self.df)

    # 인덱스에 해당하는 데이터 반환 메소드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image=cv2.imread(img_path)
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[idx,1]


        if self.transform is not None:
            image = self.transform(image)
            return image, label

# 498
from torchvision import transforms
transform = transforms.ToTensor()

dataset_train = ImageDataset(df = train, img_dir="train/", transform=transform)
dataset_valid = ImageDataset(df = valid, img_dir="train/", transform=transform)

dataset_train.__len__()
dataset_valid.__len__()




# 데이터 로더 만들기 499

from torch.utils.data import DataLoader
loader_train = DataLoader(dataset = dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=32, shuffle=False)



# 모델 만들기 501
import torch.nn as nn # 신경망 모듈 사용하기
import torch.nn.functional as F # 함수. 대문자로 표시하겠습니다.

# class
class Model(nn.Module): # 신경망 모델
    # 초기화
    def __init__(self):
        super().__init__() # nn.Model 초기화 메소드 호출
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=2)
        self.fc = nn.Linear(in_features=64*4*4, out_features=2)

    # 순전파 출력
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4)
        x = self.fc(x)
        return x




model = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)




epochs = 10 # 총 에폭
# 총 에폭만큼 반복
for epoch in range(epochs):
    epoch_loss = 0 # 에폭별 손실값 초기화
    
    # '반복 횟수'만큼 반복 
    for images, labels in loader_train:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가
        epoch_loss += loss.item() 
        # 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step()
        
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}')

In [ ]:
# 성능 검증하기
from sklearn.metrics import roc_auc_score

true_list = []
preds_list= []

model.eval()

with torch.no_grad():
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1]
        true = labels.cpu()
        preds_list.extend(preds)
        true_list.extend(true)

print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')

In [ ]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform = transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

In [ ]:
model.eval()
preds=[]
with torch.no_grad():
    for images, _ in loader_test:
        images = images.to(device)
        outputs = model(images)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        preds.extend(preds_part)